# Support vector machine

Importing packages.

In [0]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

Importing data.

In [0]:
def getdata():
  global X, Y, m, n, K
  K = 3
  data = pd.read_csv("irisdata.csv")
  X = data[["sepal length", "sepal width", "petal length", "petal width"]].to_numpy()
  m = X.shape[0]
  X = np.append(np.ones((m,1)), X, axis=1)
  Y = np.zeros((m,K))
  for i in range(m):
    if data["type"][i] == "Iris-setosa":
      Y[i][0] = 1
    if data["type"][i] == "Iris-versicolor":
      Y[i][1] = 1
    if data["type"][i] == "Iris-virginica":
      Y[i][2] = 1
  n = X.shape[1]

Performing feature scaling.

In [0]:
def scalefeatures():
  global X
  means = np.mean(X, axis=0)
  stds = np.std(X, axis=0)
  for i in range(1, X.shape[1]):
    X[:,i] = (X[:,i]-means[i])/stds[i]

Creating the training, cross-validation and test sets.

In [0]:
def createsets():
    global X, Y, Xtrain, Ytrain, Xcv, Ycv, Xtest, Ytest, mtrain, mcv, mtest
    randperm = np.random.permutation(range(m))
    X = X[randperm, :]
    Y = Y[randperm, :]
    [mtrain, mcv] = [2*m//3, m//6]
    mtest = m-mtrain-mcv
    Xtrain = X[0:mtrain, :]
    Ytrain = Y[0:mtrain, :]
    Xcv = X[mtrain:mtrain+mcv, :]
    Ycv = Y[mtrain:mtrain+mcv, :]
    Xtest = X[mtrain+mcv:m, :]
    Ytest = Y[mtrain+mcv:m, :]

Calculate the Gaussian kernel.

In [0]:
def kernel(x, y, sigma):
  return np.exp(-np.linalg.norm(x-y)**2/sigma)

Making features by creating a landmark at each training example and calculating the kernel for each sample.

In [0]:
def makefeatures(X, sigma):
  global F, Ftrain, Fcv, Ftest
  m = X.shape[0]
  F = np.eye(m)
  for i in range(m):
    for j in range(i):
      F[i][j] = kernel(X[i,:], X[j,:], sigma)
      F[j][i] = F[i][j]
  F = F[:,0:mtrain]
  Ftrain = F[0:mtrain, :]
  Fcv = F[mtrain:mtrain+mcv, :]
  Ftest = F[mtrain+mcv:m, :]

Calculating cost.

In [0]:
def cost(Theta, X, y, C):
  cost = 0
  for i in range(len(y)):
    cost += max([0, 1+(-2*y[i]+1)*X[i,:]@Theta])
  cost = C*cost + Theta[1:].T@Theta[1:]/2
  return cost

Training the SVM.

In [0]:
def trainSVM(X, Y, C, usekernel):
  opt_theta = np.empty((X.shape[1], K))
  for i in range(K):
    if usekernel:
      print("Training SVM with kernel for group ", i, ', with C=', C, ".", sep='')
    else:
      print("Training SVM without kernel for group ", i, ', with C=', C, ".", sep='')
    initial_theta = np.zeros((X.shape[1],1))
    result = minimize(cost, initial_theta, args=(X, Y[:,i], C), method='Nelder-Mead', options={'maxiter': 1000000, 'disp': False})
    if result.success:
      opt_theta[:,i] = result.x
    else:
      raise ValueError(result.message)
  return opt_theta

Making prediction.

In [0]:
def makepredictions(X, Theta):
  output = X@Theta
  predictions = np.empty((X.shape[0], K))
  for i in range(X.shape[0]):
    for j in range(K):
      predictions[i,j] = int(output[i,j]==np.amax(output[i,:]))
  return predictions

Fitting the C hyperparameter. Also fitting sigma would take too much time to run.

In [0]:
def fithyperparameters(usekernel):
  Cset = [0.1, 0.3, 1, 3, 10, 30, 100, 300]
  cv_accuracies = []
  for C in Cset:
    if usekernel:
      opt_Theta = trainSVM(Ftrain, Ytrain, C, True)
      predictions = makepredictions(Fcv, opt_Theta)
      cv_accuracies.append((Ycv.shape[0]-np.sum(np.absolute(predictions-Ycv))/2)/Ycv.shape[0])
    else:
      opt_Theta = trainSVM(Xtrain, Ytrain, C, False)
      predictions = makepredictions(Xcv, opt_Theta)
      cv_accuracies.append((Ycv.shape[0]-np.sum(np.absolute(predictions-Ycv))/2)/Ycv.shape[0])
  return Cset[cv_accuracies.index(max(cv_accuracies))]

Print accuracies.

In [0]:
def calculateaccuracies(predictions):
  train_acc = (mtrain+mcv-np.sum(np.absolute(predictions[:mtrain+mcv]-np.vstack((Ytrain,Ycv))))/2) / (mtrain+mcv)
  test_acc = (mtest-np.sum(np.absolute(predictions[mtrain+mcv:]-Ytest))/2) / mtest
  print("Training (and cv) set accuracy:", train_acc)
  print("Test set accuracy:", test_acc)

Running the algorithm.

In [12]:
getdata()
scalefeatures()
createsets()
makefeatures(X, sigma=1)
C_nokernel = fithyperparameters(usekernel=False)
C_kernel = fithyperparameters(usekernel=True)
opt_theta_nokernel = trainSVM(np.vstack((Xtrain,Xcv)), Ytrain, C_nokernel, False)
opt_theta_kernel = trainSVM(np.vstack((Ftrain,Fcv)), Ytrain, C_kernel, True)
predictions_nokernel = makepredictions(X, opt_theta_nokernel)
predictions_kernel = makepredictions(F, opt_theta_kernel)
print("\nPerformance without kernel:")
calculateaccuracies(predictions_nokernel)
print("\nPerfomance with kernel:")
calculateaccuracies(predictions_kernel)

Training SVM without kernel for group 0, with C=0.1.
Training SVM without kernel for group 1, with C=0.1.
Training SVM without kernel for group 2, with C=0.1.
Training SVM without kernel for group 0, with C=0.3.
Training SVM without kernel for group 1, with C=0.3.
Training SVM without kernel for group 2, with C=0.3.
Training SVM without kernel for group 0, with C=1.
Training SVM without kernel for group 1, with C=1.
Training SVM without kernel for group 2, with C=1.
Training SVM without kernel for group 0, with C=3.
Training SVM without kernel for group 1, with C=3.
Training SVM without kernel for group 2, with C=3.
Training SVM without kernel for group 0, with C=10.
Training SVM without kernel for group 1, with C=10.
Training SVM without kernel for group 2, with C=10.
Training SVM without kernel for group 0, with C=30.
Training SVM without kernel for group 1, with C=30.
Training SVM without kernel for group 2, with C=30.
Training SVM without kernel for group 0, with C=100.
Training SV